In [38]:
%load_ext autoreload
%autoreload 2

import docker
import logging
from jtlutil.docker.dctl import *
import hashlib

from jtlutil.config import get_config

config = get_config('./config.env')

logging.basicConfig(level=logging.ERROR)
logger.setLevel(logging.DEBUG)

client = docker.DockerClient(base_url=config.SSH_URI )

print(config.SSH_URI)
ensure_network_exists(client, 'x11', is_external=False, network_type="bridge")
ensure_network_exists(client, 'jtlctl', is_external=False, network_type="bridge")
ensure_network_exists(client, 'caddy', is_external=True, network_type="bridge")

config.KST_REPORTING_URL

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


DEBUG:jtlutil.docker.dctl:Network 'x11' already exists.


ssh://root@64.23.135.161


DEBUG:jtlutil.docker.dctl:Network 'jtlctl' already exists.
DEBUG:jtlutil.docker.dctl:Network 'caddy' already exists.


'http://cs-spawner:8000/telem'

In [8]:
def make_servers(usernames):
    servers = []
    for username in usernames:
        nvc, pa = create_cs_pair(client, config, config.IMAGES_PYTHONCS, username)
        yield (nvc, pa)
    return servers

print('----')
for nvc, pa in make_servers(['jed','osvaldo', 'eric']):
    print(nvc.id, pa.labels['caddy'])


INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/novnc-service:v0.1.3'...


----


INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 81b081a8f85991990d1603f8af7c48603159f741149d68f5d44df36b8f21fc36
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : fc7f9235fcdc4f062e1429e10918fa9491398caade111e528450db7c06a7c660
INFO:jtlutil.docker.dctl:Connecting container to network 'caddy' : 341c37b26522cf9aaf53c4e37df9a6b94fe50cb0cc79e05d7d35d6a3f2327fb5
INFO:jtlutil.docker.dctl:Container 'jed-novnc' (b47d643d6ed75d649cfa4dab066cd7a1e24795d97609c9a575b942d884f24aba) created successfully.
INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/league-infrastructure/docker-codeserver-python:v0.1.5'...
INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 81b081a8f85991990d1603f8af7c48603159f741149d68f5d44df36b8f21fc36
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : fc7f9235fcdc4f062e1429e10918fa9491398caade111e528450db7c06a7c660
INFO:jtlutil.docker.dctl:Connecting container to network 'cadd

b47d643d6ed75d649cfa4dab066cd7a1e24795d97609c9a575b942d884f24aba jed.do.jointheleague.org


INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 81b081a8f85991990d1603f8af7c48603159f741149d68f5d44df36b8f21fc36
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : fc7f9235fcdc4f062e1429e10918fa9491398caade111e528450db7c06a7c660
INFO:jtlutil.docker.dctl:Connecting container to network 'caddy' : 341c37b26522cf9aaf53c4e37df9a6b94fe50cb0cc79e05d7d35d6a3f2327fb5
INFO:jtlutil.docker.dctl:Container 'osvaldo-novnc' (4413bc4bd0f4c610a987b068fadb49564207a2e6ae9ff69efda744f7b4f7e054) created successfully.
INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/league-infrastructure/docker-codeserver-python:v0.1.5'...
INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 81b081a8f85991990d1603f8af7c48603159f741149d68f5d44df36b8f21fc36
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : fc7f9235fcdc4f062e1429e10918fa9491398caade111e528450db7c06a7c660
INFO:jtlutil.docker.dctl:Connecting container to network '

4413bc4bd0f4c610a987b068fadb49564207a2e6ae9ff69efda744f7b4f7e054 osvaldo.do.jointheleague.org


INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 81b081a8f85991990d1603f8af7c48603159f741149d68f5d44df36b8f21fc36
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : fc7f9235fcdc4f062e1429e10918fa9491398caade111e528450db7c06a7c660
INFO:jtlutil.docker.dctl:Connecting container to network 'caddy' : 341c37b26522cf9aaf53c4e37df9a6b94fe50cb0cc79e05d7d35d6a3f2327fb5
INFO:jtlutil.docker.dctl:Container 'eric-novnc' (05528f62ac6f6fc540c46e5a138d4a84dadc183453c245dad11dcbba446ee28f) created successfully.
INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/league-infrastructure/docker-codeserver-python:v0.1.5'...
INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 81b081a8f85991990d1603f8af7c48603159f741149d68f5d44df36b8f21fc36
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : fc7f9235fcdc4f062e1429e10918fa9491398caade111e528450db7c06a7c660
INFO:jtlutil.docker.dctl:Connecting container to network 'cad

05528f62ac6f6fc540c46e5a138d4a84dadc183453c245dad11dcbba446ee28f eric.do.jointheleague.org


In [39]:
list(make_servers(['jed']))

containers = client.containers.list(filters={"label": f"jtl"}, all=True)

for c in containers:
    print(c.id, c.status, c.labels['caddy'])

INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/novnc-service:v0.1.3'...
DEBUG:jtlutil.docker.dctl:Port bindings: None
DEBUG:jtlutil.docker.dctl:Volumes: None
INFO:jtlutil.docker.dctl:Connecting container to network 'x11' : 70fb967b471875d0782b3bfc9d4cf106fb4a653665127f127f76ba744c328a72
INFO:jtlutil.docker.dctl:Connecting container to network 'jtlctl' : faf019272add9022b098afa197c53577b7d3f28d4e8b8afb32ae42850a7d6156
INFO:jtlutil.docker.dctl:Connecting container to network 'caddy' : 0b503a35a820d17b7a65b953195b8e2bd3c3a63bd55bc3b91cd2e4262e5d26e1
INFO:jtlutil.docker.dctl:Container 'jed-novnc' (67bb70a1b30f450ce12210b4c77887e08e473250d91e5604b4a42a6570e18f97) created successfully.
INFO:jtlutil.docker.dctl:Creating container from image 'ghcr.io/league-infrastructure/league-infrastructure/docker-codeserver-python:v0.1.5'...
DEBUG:jtlutil.docker.dctl:Port bindings: {'8080/tcp': None}
DEBUG:jtlutil.docker.dctl:Volumes: None
INFO:jtlutil.docker.dctl:Con

45474469767d4f8baf33a9c2cac29f9329f6b0bc1bd84dade3ccdd7ba03cbc5b running jed.do.jointheleague.org
67bb70a1b30f450ce12210b4c77887e08e473250d91e5604b4a42a6570e18f97 running jed-novnc.do.jointheleague.org
86fdd94433889853efc825ea18276adbd876e0fa420a84a865d2a76c5556999c running eric-busboom-org.do.jointheleague.org
e779f1d0d6867149e6f6c26e588b7f0050560638ae30b983d8528da0564fe2ee running eric-busboom-org-novnc.do.jointheleague.org
48257f3941161cfad1155fc6f3a82c699bcefe427dc2d9118f183ff99140c186 running test-user-jointheleague-org.do.jointheleague.org
619b73f9801d1e32bedb2e9cc67f27d56170de64a55108f499dc83ffd7987d82 running test-user-jointheleague-org-novnc.do.jointheleague.org
15727cf7cee9e182127a34f9de291d80e3601328031609990696b7b92bba64cc running eric-busboom-jointheleague-org.do.jointheleague.org
7dce77acf7458dd16ea6aac66cf7e1d12fea8b1d4678d74f7ee834af8c0e4b59 running eric-busboom-jointheleague-org-novnc.do.jointheleague.org


In [36]:

containers = client.containers.list(filters={"label": f"jtl"}, all=True)

for c in containers:
    print(c.id, c.status, c.labels['caddy'])
    if c.status == 'running':
        try:
            c.stop()
            c.remove()
        except Exception as e:
            print(e)
            

In [5]:
container_status(client, 'eric-busboom-org')

'non-exist'

In [44]:
id = '86fdd94433889853efc825ea18276adbd876e0fa420a84a865d2a76c5556999c'
c = client.containers.get(id)

for e in c.diff():
    print(e)
    